## 08. 분류 실습 - 캐글 산탄데르 고객 만족 예측

- 레이블 명 TARGET, 1: 불만, 0: 만족
- 1이 많은 불균일 데이터 라서 accuracy score 보다는 roc-auc 가 평가 성능 지표로 적절

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

cust_df = pd.read_csv("data/train.csv", encoding = "latin-1")
print("dataset shape: ", cust_df.shape)
cust_df.head(3)

dataset shape:  (76020, 371)


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.17,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.03,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.77,0


In [2]:
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


In [3]:
print(cust_df["TARGET"].value_counts())
unsatisfied_cnt = cust_df[cust_df["TARGET"] == 1].TARGET.count()
total_cnt = cust_df.TARGET.count()
print("unsatisfied / total = ", unsatisfied_cnt / total_cnt)

0    73012
1     3008
Name: TARGET, dtype: int64
unsatisfied / total =  0.0395685345961589


In [4]:
cust_df.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,...,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,43781.947379,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,...,455.887218,113.959637,783.207399,538.439211,2013.125393,147.786584,4040.337842,2852.579397,1.826646e+05,0.194945
min,1.000000,-999999.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.163750e+03,0.000000
25%,38104.750000,2.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.787061e+04,0.000000
50%,76043.000000,2.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.064092e+05,0.000000
75%,113748.750000,2.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.187563e+05,0.000000
max,151838.000000,238.000000,105.000000,210000.000000,12888.030000,21024.810000,8237.820000,11073.570000,6600.000000,6600.000000,...,50003.880000,20385.720000,138831.630000,91778.730000,438329.220000,24650.010000,681462.900000,397884.300000,2.203474e+07,1.000000


In [5]:
print("cust_df[\"var3\"].value_counts()[:10]\n", cust_df["var3"].value_counts()[:10])
# -999999 특정 예외 값을 나타내는데 편차가 심해서 가장 값이 많은 2로 바꾸자


cust_df["var3"].value_counts()[:10]
  2         74165
 8           138
-999999      116
 9           110
 3           108
 1           105
 13           98
 7            97
 4            86
 12           85
Name: var3, dtype: int64


In [6]:
cust_df["var3"].replace(-999999, 2, inplace = True)
cust_df.drop("ID", axis = 1, inplace = True)

print("cust_df[\"var3\"].value_counts()[:10]\n", cust_df["var3"].value_counts()[:10])
cust_df.head(3)


cust_df["var3"].value_counts()[:10]
 2     74281
8       138
9       110
3       108
1       105
13       98
7        97
4        86
12       85
6        82
Name: var3, dtype: int64


,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.17,0
1,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.03,0
2,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.77,0


In [7]:
from sklearn.model_selection import train_test_split

X_features = cust_df.iloc[:, :-1]
y_labels = cust_df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size = 0.2, random_state = 0)

print(f"train: {X_train.shape}, test: {X_test.shape}")
print(y_train.count())

print(f"train\n {y_train.value_counts()/ y_train.count()}")
print(f"test\n {y_test.value_counts()/ y_test.count()}")
# 학습, 테스트 데이터 라벨 비율이 비슷하게 만들어짐


train: (60816, 369), test: (15204, 369)
60816
train
 0    0.960964
1    0.039036
Name: TARGET, dtype: float64
test
 0    0.9583
1    0.0417
Name: TARGET, dtype: float64


### XGBoost 모델 학습과 하이퍼 파라미터 튜닝

- 평가 데이터 세트로 테스트 데이터 세트 사용하면 과적합 가능성 증가함


In [8]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

xgb_clf = XGBClassifier(n_estimators = 250, random_state = 156)

xgb_clf.fit(X_train, y_train, early_stopping_rounds = 50, eval_metric = "auc", eval_set = [(X_train, y_train), (X_test, y_test)], verbose = 15)

xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:, 1], average = "macro")
print(f"ROC AUC: {xgb_roc_score}")


[0]	validation_0-auc:0.82005	validation_1-auc:0.81157
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 50 rounds.
[15]	validation_0-auc:0.88072	validation_1-auc:0.84117
[30]	validation_0-auc:0.89738	validation_1-auc:0.83959
[45]	validation_0-auc:0.90767	validation_1-auc:0.83693
[60]	validation_0-auc:0.91762	validation_1-auc:0.83576
Stopping. Best iteration:
[14]	validation_0-auc:0.87799	validation_1-auc:0.84135

ROC AUC: 0.8413487915404584


In [9]:
from sklearn.model_selection import GridSearchCV

xgb_clf = XGBClassifier(n_estimators = 50)
params = {
    "max_depth": [5, 7],
    "min_child_weight":[1, 3],
    "colsample_bytree":[0.5, 0.75]
}

gridcv = GridSearchCV(xgb_clf, param_grid = params, cv = 2)
gridcv.fit(X_train, y_train, early_stopping_rounds = 15, eval_metric= "auc", eval_set = [(X_train, y_train), (X_test, y_test)], verbose = 15)

print("best parmas: ", gridcv.best_params_)
xgb_roc_score = roc_auc_score(y_test, gridcv.predict_proba(X_test)[:, 1], average = "macro")
print("ROC AUC: ", xgb_roc_score)



[0]	validation_0-auc:0.71214	validation_1-auc:0.71487
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 15 rounds.
[15]	validation_0-auc:0.85352	validation_1-auc:0.83359
[30]	validation_0-auc:0.86454	validation_1-auc:0.83584
[45]	validation_0-auc:0.86860	validation_1-auc:0.83482
[49]	validation_0-auc:0.86984	validation_1-auc:0.83489
[0]	validation_0-auc:0.69509	validation_1-auc:0.69896
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 15 rounds.
[15]	validation_0-auc:0.85501	validation_1-auc:0.83771
Stopping. Best iteration:
[12]	validation_0-auc:0.85000	validation_1-auc:0.84021

[0]	validation_0-auc:0.71220	validation_1-auc:0.71486
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 15 rounds.
[15]	valid

- GridSearchCV 로 구한 최적 파라미터와 다른 파라미터를 추가해서 더 많은 파라미터를 최적화 시킬 수 있다.
- fit한 classifier 를 plot importance() 로 피처 중요도 시각화 가능


### LightGBM 모델 학습과 하이퍼 파라미터 튜닝



In [10]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators = 500)
evals = [(X_test, y_test)]

lgbm_clf.fit(X_train, y_train, early_stopping_rounds = 100, eval_metric = "auc", eval_set = evals, verbose = 15)
lgbm_roc_score = roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:, 1], average = "macro")
print("ROC AUC: ", lgbm_roc_score)


Training until validation scores don't improve for 100 rounds
[15]	valid_0's auc: 0.840928	valid_0's binary_logloss: 0.14161
[30]	valid_0's auc: 0.839196	valid_0's binary_logloss: 0.139641
[45]	valid_0's auc: 0.839703	valid_0's binary_logloss: 0.139445
[60]	valid_0's auc: 0.838319	valid_0's binary_logloss: 0.13969
[75]	valid_0's auc: 0.83772	valid_0's binary_logloss: 0.13992
[90]	valid_0's auc: 0.836752	valid_0's binary_logloss: 0.140225
[105]	valid_0's auc: 0.836752	valid_0's binary_logloss: 0.140391
Early stopping, best iteration is:
[15]	valid_0's auc: 0.840928	valid_0's binary_logloss: 0.14161
ROC AUC:  0.8409281094855902


In [11]:
# 하이퍼 파라미터 튜닝

lgbm_clf = LGBMClassifier(n_estimators = 100)
params = {
    "num_leaves": [32, 64],
    "max_depth": [128, 160],
    "min_child_samples": [60, 100],
    "subsample": [0.8, 1]
}
gridcv = GridSearchCV(lgbm_clf, param_grid = params, cv = 2)
gridcv.fit(X_train, y_train, early_stopping_rounds = 15, eval_metric = "auc", eval_set = [(X_train, y_train), (X_test, y_test)], verbose = 30)

print("best parmas: ", gridcv.best_params_)
lgbm_roc_score = roc_auc_score(y_test, gridcv.predict_proba(X_test)[:, 1], average = "macro")
print("ROC AUC: ", lgbm_roc_score)


Training until validation scores don't improve for 15 rounds
Early stopping, best iteration is:
[8]	valid_0's auc: 0.845435	valid_0's binary_logloss: 0.137252	valid_1's auc: 0.836312	valid_1's binary_logloss: 0.147155
Training until validation scores don't improve for 15 rounds
[30]	valid_0's auc: 0.86812	valid_0's binary_logloss: 0.125364	valid_1's auc: 0.836871	valid_1's binary_logloss: 0.139878
Early stopping, best iteration is:
[25]	valid_0's auc: 0.864708	valid_0's binary_logloss: 0.126629	valid_1's auc: 0.837921	valid_1's binary_logloss: 0.140053
Training until validation scores don't improve for 15 rounds
Early stopping, best iteration is:
[8]	valid_0's auc: 0.845435	valid_0's binary_logloss: 0.137252	valid_1's auc: 0.836312	valid_1's binary_logloss: 0.147155
Training until validation scores don't improve for 15 rounds
[30]	valid_0's auc: 0.86812	valid_0's binary_logloss: 0.125364	valid_1's auc: 0.836871	valid_1's binary_logloss: 0.139878
Early stopping, best iteration is:
[25]	

In [12]:
lgbm_clf = LGBMClassifier(n_estimators = 1000, num_leaves = 32, subsample = 0.8, min_child_samples = 100, max_depth = 128)
evals = [(X_test, y_test)]
lgbm_clf.fit(X_train, y_train, early_stopping_rounds = 100, eval_metric = "auc", eval_set = evals, verbose = 15)
lgbm_roc_score = roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:, 1], average = "macro")

print("ROC AUC: ", lgbm_roc_score)


Training until validation scores don't improve for 100 rounds
[15]	valid_0's auc: 0.84154	valid_0's binary_logloss: 0.141254
[30]	valid_0's auc: 0.838922	valid_0's binary_logloss: 0.139324
[45]	valid_0's auc: 0.838529	valid_0's binary_logloss: 0.139115
[60]	valid_0's auc: 0.838145	valid_0's binary_logloss: 0.13923
[75]	valid_0's auc: 0.837359	valid_0's binary_logloss: 0.139475
[90]	valid_0's auc: 0.836314	valid_0's binary_logloss: 0.139907
[105]	valid_0's auc: 0.834578	valid_0's binary_logloss: 0.140374
Early stopping, best iteration is:
[11]	valid_0's auc: 0.841659	valid_0's binary_logloss: 0.14327
ROC AUC:  0.8416587279076968
